# Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gymnasium as gym
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
import stable_baselines3
import gym_trading_env
from gym_trading_env.renderer import Renderer   
from pathlib import Path
from BitcoinDownloader import download_exchange_data, get_dataframes
#from BitcoinIndicators import Indicators
from ta_indicators import Indicators
from BitcoinRewards import reward_function
from BitcoinDynamicFeatures import dynamic_feature_last_position_taken, dynamic_feature_real_position
from tqdm import tqdm

import itertools

import torch
torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

# Download Data if needed

In [3]:
download_again = False
data_path = './data/binance-BTCUSDT-1h.pkl'
if not Path(data_path).is_file() or download_again:
    download_exchange_data()
else:
    print("data already downloaded")


download_again = False
data_path = './data/bitfinex2-BTCUSDT-1h.pkl'
if not Path(data_path).is_file() or download_again:
    download_exchange_data(data_path=['bitfinex2'])
else:
    print("data already downloaded")


download_again = False
data_path = './data/huobi-BTCUSDT-1h.pkl'
if not Path(data_path).is_file() or download_again:
    download_exchange_data(data_path=['huobi'])
else:
    print("data already downloaded")

data already downloaded
data already downloaded
data already downloaded


In [4]:
bin_train_df, bin_eval_df = get_dataframes(data_path='./data/binance-BTCUSDT-1h.pkl')
bitf_train_df, bitf_eval_df = get_dataframes(data_path='./data/bitfinex2-BTCUSDT-1h.pkl')
huobi_train_df, huobi_eval_df = get_dataframes(data_path='./data/huobi-BTCUSDT-1h.pkl')

# Create features

### Create Features for Training Sets

In [5]:
bin_path = './data/binance_indicators.csv'
bitf_path = './data/bitfinex2_indicators.csv'
huobi_path = './data/huobi_indicators.csv'

# Process training sets
bin_train_df.to_csv(bin_path)
bin_indicators = Indicators(bin_path)

bitf_train_df.to_csv(bitf_path)
bitf_indicators = Indicators(bitf_path)

huobi_train_df.to_csv(huobi_path)
huobi_indicators = Indicators(huobi_path)

"""training_df.to_csv(ind_path)
indicators = Indicators(ind_path)
indicators.to_csv(ind_path)
training_df = pd.read_csv(ind_path)
training_df["date_open"] = pd.to_datetime(training_df["date_open"])
training_df.set_index("date_open", inplace=True)

training_df.dropna(inplace=True)
print(training_df.head(3))
print(training_df.tail(3))"""

d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f"DX_{period}"] = (
d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f"feature_ADX_{period}"] = (
d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:254: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

KeyboardInterrupt: 

### Create Features for Eval Sets

In [6]:
bin_eval_path = './data/binance_eval_indicators.csv'
bitf_eval_path = './data/bitfinex2_eval_indicators.csv'
huobi_eval_path = './data/huobi_eval_indicators.csv'

# Process eval sets
bin_eval_df.to_csv(bin_eval_path)
bin_eval_indicators = Indicators(bin_eval_path)

"""bitf_eval_df.to_csv(bitf_path)
bitf_eval_indicators = Indicators(bitf_eval_path)

huobi_eval_df.to_csv(huobi_path)
huobi_eval_indicators = Indicators(huobi_eval_path)"""

d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:219: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f"DX_{period}"] = (
d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f"feature_ADX_{period}"] = (
d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:254: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calli

'bitf_eval_df.to_csv(bitf_path)\nbitf_eval_indicators = Indicators(bitf_eval_path)\n\nhuobi_eval_df.to_csv(huobi_path)\nhuobi_eval_indicators = Indicators(huobi_eval_path)'

### Write all datasets to pkl files

Training environment will use directory of all three pkl files.

In [7]:
#bin_indicators.to_pickle('./data/train/binance.pkl')
#bitf_indicators.to_pickle('./data/train/bitfinex.pkl')
#huobi_indicators.to_pickle('./data/train/huobi.pkl')


bin_eval_indicators.to_pickle('./data/eval/binance.pkl')
#bitf_eval_indicators.to_pickle('./data/eval/bitfinex.pkl')
#huobi_eval_indicators.to_pickle('./data/eval/huobi.pkl')


d:\Google Drive\A1-DOCUMENTS\A1_OSU\CS467 - Capstone\Code\ai-bitcoin-bot\ta_indicators.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.indicators.dropna(inplace=True)


### Create single dataframe for eval dataset

Eval environment will just use a single dataframe

In [8]:
eval_df = pd.read_pickle('./data/eval/binance.pkl')

eval_df.head()

,close,open,high,low,volume,feature_ROC_2,feature_ROC_4,feature_ROC_6,feature_ROC_8,feature_ROC_16,...,feature_RSI_16,feature_RSI_32,feature_RSI_64,feature_ATR_2,feature_ATR_4,feature_ATR_6,feature_ATR_8,feature_ATR_16,feature_ATR_32,feature_ATR_64
date_open,,,,,,,,,,,,,,,,,,,,,
2023-01-06 08:00:00,16802.11,16794.33,16812.22,16791.47,5475.13940,0.087678,-0.162814,-0.252048,-0.349053,-0.119872,...,29.803864,32.066543,36.547572,21.735,27.8675,26.600000,30.04500,33.138125,34.421563,45.555625
2023-01-06 09:00:00,16787.10,16802.11,16802.27,16762.50,7581.99164,-0.027633,-0.136706,-0.275297,-0.320604,-0.143718,...,30.591028,31.149886,36.091207,30.260,29.9325,30.633333,29.64375,33.432500,34.343438,45.572031
2023-01-06 10:00:00,16767.06,16787.10,16802.98,16760.00,6268.17791,-0.204559,-0.114143,-0.319608,-0.390071,-0.249449,...,24.957208,30.065782,35.528952,41.375,31.5550,32.370000,30.29375,33.175625,35.025938,45.825938
2023-01-06 11:00:00,16738.22,16767.15,16772.78,16709.88,10651.25261,-0.296271,-0.277148,-0.399186,-0.506780,-0.424402,...,18.533310,29.253682,35.015966,52.940,41.6000,37.601667,36.21000,34.055000,36.375625,46.259844
2023-01-06 12:00:00,16731.02,16738.74,16749.00,16717.66,7194.28857,-0.223641,-0.404734,-0.374106,-0.560501,-0.562344,...,18.379364,28.460866,34.588153,47.120,44.2475,36.743333,36.05750,34.333125,36.678750,46.123906


# Create Environment

In [8]:
"""training_env = gym.make("TradingEnv",
        name= "BTCUSD",
        df = training_df, # Your dataset with your custom features
        positions = [0, 1], # -1 (=SHORT), 0(=SELL ALL), +1 (=BUY ALL)
        #trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        #borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
        #dynamic_feature_functions = [dynamic_features]
        # reward_function = reward_function,
        portfolio_initial_value = 1000,
        reward_function = reward_function
        #max_episode_duration = 1000,
    )"""


training_env = gym.make("MultiDatasetTradingEnv",
        name= "BTCUSD",
        dataset_dir= './data/train/*.pkl', # Your dataset with your custom features
        positions = [0, 1], # -1 (=SHORT), 0(=SELL ALL), +1 (=BUY ALL)
        #trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        #borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
        dynamic_feature_functions = [dynamic_feature_last_position_taken, dynamic_feature_real_position],
        portfolio_initial_value = 1000,
        reward_function = reward_function,
        initial_position = 0,
        #max_episode_duration = 1000,
)

observation, info = training_env.reset()
print(observation, info)

[   2.111697     3.1281855    6.2386966    7.783201    -3.7939115
   -8.029003    -4.6376657   35.867607   -45.050323  -205.62448
 -174.26988     58.428787  -284.2601     -48.21791    100.
  100.          79.94675     40.692715    57.1144      64.7262
   40.741234    50.          43.443478    42.622845    46.772453
   31.911493    31.503231    32.214336   100.485      106.1775
   91.598335   134.965      143.4325     111.090935    82.007965
    0.           0.       ] {'idx': 0, 'step': 0, 'date': numpy.datetime64('2017-08-22T12:00:00.000000000'), 'position_index': 0, 'position': 0, 'real_position': 0, 'data_low': 3853.53, 'data_volume': 75.787584, 'data_close': 3934.01, 'data_open': 3866.48, 'data_high': 3990.12, 'portfolio_valuation': 1000.0, 'portfolio_distribution_asset': 0, 'portfolio_distribution_fiat': 1000.0, 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0, 'portfolio_distribution_interest_fiat': 

# Create Model

In [9]:
model = RecurrentPPO('MlpLstmPolicy', # feed-forward neural network with multiple hidden layers
            training_env, # environment in which the agent interacts and learns
            verbose=1, # enables the training progress to be printed during the learning process
            gamma=0.97, # determines the importance of future rewards compared to immediate rewards
            n_steps=5, # steps to collect samples from the environment before performing an update
            ent_coef=0.02, # encourages exploration by adding entropy to the policy loss
            learning_rate=0.0001, # controls the step size at which model's parameters are updated based on the gradient of the loss function
            clip_range=0.2, # limits the update to a certain range to prevent large policy updates
            device="cuda" if torch.cuda.is_available() else "cpu",
            )
#n = len(training_df)


NameError: name 'training_env' is not defined

### Train Model (can take a while)

In [10]:
model.learn(total_timesteps=10000)

---------------------------
| time/              |    |
|    fps             | 23 |
|    iterations      | 1  |
|    time_elapsed    | 0  |
|    total_timesteps | 5  |
---------------------------
--------------------------------------------
| time/                   |                |
|    fps                  | 20             |
|    iterations           | 2              |
|    time_elapsed         | 0              |
|    total_timesteps      | 10             |
| train/                  |                |
|    approx_kl            | 0.000108468535 |
|    clip_fraction        | 0              |
|    clip_range           | 0.2            |
|    entropy_loss         | -0.693         |
|    explained_variance   | -0.99          |
|    learning_rate        | 0.0001         |
|    loss                 | -0.023         |
|    n_updates            | 10             |
|    policy_gradient_loss | -0.00562       |
|    value_loss           | 0.0373         |
---------------------------------------

In [11]:
model.save('./models/RPPO_totalSteps=10k_nsteps=5_LR=0.0003_epochs=10')

Exception ignored in: <_io.FileIO name='models\\RPPO_totalSteps=10k_nsteps=5_LR=0.0003_epochs=10' mode='wb' closefd=True>
Traceback (most recent call last):
  File "d:\Python\Lib\site-packages\stable_baselines3\common\base_class.py", line 845, in save
    save_to_zip_file(path, data=data, params=params_to_save, pytorch_variables=pytorch_variables)


### Load Model (if applicable)

In [10]:
model = RecurrentPPO.load('./models/RPPO_totalSteps=10k_nsteps=144_LR=0.0003_epochs=10')

### This confirms that the previously-trained model has been loaded

ep_info_buffer should be non-null

In [13]:
model.policy

RecurrentActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=256, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=256, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=2, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
  (lstm_actor): LSTM(37, 256)
  (lstm_critic): LSTM(37, 256)
)

In [11]:
model.ep_info_buffer

deque([], maxlen=100)

# Calculate indicators for evaluation dataset

In [12]:
eval_env = gym.make("TradingEnv",
        name= "BTCUSD",
        df = eval_df, # Your dataset with your custom features
        positions = [0, 1], # -1 (=SHORT), 0(=SELL ALL), +1 (=BUY ALL)
        portfolio_initial_value = 1000,
        reward_function = reward_function,
        dynamic_feature_functions = [dynamic_feature_last_position_taken, dynamic_feature_real_position],
        initial_position = 0,
    )

observation, info = eval_env.reset()
print(observation.shape)
print(observation)
print(info)

(37,)
[ 8.7677605e-02 -1.6281354e-01 -2.5204769e-01 -3.4905249e-01
 -1.1987240e-01 -3.3908841e-01  6.7657948e-01 -1.3897646e+02
  5.9759155e+01  1.0690672e+02 -3.5503836e+02 -1.1604339e+02
 -1.4387981e+02 -4.3159134e+01  9.4134003e+01  8.7169312e+01
  8.2858330e+01  8.9389130e+01  8.5556389e+01  7.5845436e+01
  6.8625328e+01  5.0000000e+01  2.2887589e+01  1.8897150e+01
  1.8681402e+01  2.9803864e+01  3.2066544e+01  3.6547573e+01
  2.1735001e+01  2.7867500e+01  2.6600000e+01  3.0045000e+01
  3.3138126e+01  3.4421562e+01  4.5555626e+01  0.0000000e+00
  0.0000000e+00]
{'idx': 0, 'step': 0, 'date': numpy.datetime64('2023-01-06T08:00:00.000000000'), 'position_index': 0, 'position': 0, 'real_position': 0, 'data_open': 16794.33, 'data_close': 16802.11, 'data_high': 16812.22, 'data_volume': 5475.1394, 'data_low': 16791.47, 'portfolio_valuation': 1000.0, 'portfolio_distribution_asset': 0, 'portfolio_distribution_fiat': 1000.0, 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_

# Test trained model on testing data

In [13]:
for _ in tqdm(range(len(eval_df))):
    position_index, _states = model.predict(observation)
    observation, reward, done, truncated, info = eval_env.step(position_index)
    eval_env.save_for_render(dir = "render_logs")
    if done or truncated:
        break

100%|█████████▉| 5119/5121 [05:50<00:00, 14.59it/s]

Market Return : 72.49%   |   Portfolio Return : 55.03%   |   


# Render results

In [14]:
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Aug/2023 18:37:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2023 18:37:42] "GET /update_data/BTCUSD_2023-08-14_18-37-26.pkl HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2023 18:37:42] "GET /metrics HTTP/1.1" 200 -
